<a href="https://colab.research.google.com/github/Degananda264/Amazon-reviews-analysis/blob/master/Amazon_reviews_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [0]:
df=pd.read_csv(r"/content/reviews.csv")

In [7]:
df.shape


(104975, 3)

In [8]:
df.head()

,Unnamed: 0,text,sentiment
0,0,got them fast. i burned about six discs right ...,1
1,1,"Well, it fits my nano 4 quite well. And the pr...",1
2,2,"The battery arrived on the promised date, and ...",1
3,3,"Bought this one from Amazon, sold by the MiniS...",1
4,4,"So i ordered 3 chargers and surprisingly, they...",0


In [0]:
df.drop("Unnamed: 0",axis=1,inplace=True)

In [10]:
df.head()

,text,sentiment
0,got them fast. i burned about six discs right ...,1
1,"Well, it fits my nano 4 quite well. And the pr...",1
2,"The battery arrived on the promised date, and ...",1
3,"Bought this one from Amazon, sold by the MiniS...",1
4,"So i ordered 3 chargers and surprisingly, they...",0


In [11]:
df.dtypes

text         object
sentiment     int64
dtype: object

In [12]:
df[df['text'].isnull()==True]


,text,sentiment
77812,NaN,1


In [0]:
df=df.drop(77812)


In [14]:
df[df['text'].isnull()==True]

,text,sentiment


In [0]:
df=df.reset_index()

In [0]:
df.drop('index',axis=1,inplace=True)

In [17]:
df.head()

,text,sentiment
0,got them fast. i burned about six discs right ...,1
1,"Well, it fits my nano 4 quite well. And the pr...",1
2,"The battery arrived on the promised date, and ...",1
3,"Bought this one from Amazon, sold by the MiniS...",1
4,"So i ordered 3 chargers and surprisingly, they...",0


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model,Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop,adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import tensorflow as tf

In [0]:
X=df.text
y=df.sentiment

In [20]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
import tensorflow_datasets as tfds
tokenizer=tfds.features.text.Tokenizer()
vocabulary_set=set()
for i in X:
  review_tokens=tokenizer.tokenize(i)
  vocabulary_set.update(review_tokens)



In [0]:
vocabulary_size=len(vocabulary_set)

In [27]:
print(vocabulary_size)

73738


In [0]:
max_words = vocabulary_size
max_len = 128
# using Tokenizer
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X)
sequences = tok.texts_to_sequences(X)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
X_train,X_test,Y_train,Y_test = train_test_split(sequences_matrix,y,test_size=0.20)

In [104]:
lstm_model = Sequential(name = 'lstm_nn_model')
lstm_model.add(layer = Embedding(vocabulary_size,128))
lstm_model.add(layer = LSTM(units = 120, dropout = 0.2, recurrent_dropout = 0.2, name = '2nd_layer'))
lstm_model.add(layer = Dropout(rate = 0.5, name = '3rd_layer'))
lstm_model.add(layer = Dense(units = 120,  activation = 'relu', name = '4th_layer'))
lstm_model.add(layer = Dropout(rate = 0.5, name = '5th_layer'))
lstm_model.add(layer = Dense(units = 1,  activation = 'sigmoid', name = 'output_layer'))
# compiling the model
lstm_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
lstm_model.fit(X_train,Y_train,batch_size=128,epochs=10,validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 67183 samples, validate on 16796 samples
Epoch 1/10
67183/67183 [==============================] - 271s 4ms/step - loss: 0.3831 - accuracy: 0.8325 - val_loss: 0.2971 - val_accuracy: 0.8762
Epoch 2/10
67183/67183 [==============================] - 260s 4ms/step - loss: 0.2793 - accuracy: 0.8907 - val_loss: 0.3033 - val_accuracy: 0.8737


In [106]:
test_loss, test_acc = lstm_model.evaluate(X_test,Y_test)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))


20995/20995 [==============================] - 19s 911us/step
Test Loss: 0.31297980979480655
Test Accuracy: 0.8720647692680359


In [0]:
lstm_model.save('/content/model.hdf5')

In [44]:
new_model=tf.keras.models.load_model('/content/model.hdf5')

In [0]:
pred_text='color and dial doest not look as advertised. Shipping was not any better as well'
pred_data=pd.Series(pred_text)



In [99]:
type(pred_data[0])

str

In [0]:
# using Tokenizer

tok.fit_on_texts(pred_data)
sequences = tok.texts_to_sequences(pred_data)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [101]:
sequences_matrix

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,   200,     4,  1798, 15269,
           17,   223,    22,   483,   279,    20,    17,   100,   106,
      

In [0]:
predictions=new_model.predict(sequences_matrix)

In [103]:
predictions

array([[0.40645412]], dtype=float32)